**Sentiment Analysis**

**I. Individual Analysis**

**Kitchen**

*General Imports*

In [1]:
import nltk
from random import shuffle
from sklearn.linear_model import LogisticRegression
from nltk.metrics.scores import (precision, recall, accuracy)
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import RandomForestClassifier

**Features Construction**

Useful functions to load the different features models

In [2]:
def load_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = int(theword[1])
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [3]:
def load_binary_bow(thefeatures,filename):
    '''
    Loads the tags in the filename and appends them to the
    features list. Filters them to change it to 1 for each 
    existent element in the Bag of Words.
    '''
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats = {}
        for word in range(len(splited)-1):
            theword = splited[word].split(":")
            feats[theword[0]] = 1
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats,tag))
        line = f.readline()
    f.close()

In [4]:
def load_lexicon(thefeatures,filename):
    '''
    Loads the tags process them to create the Features based on the
    AFINN-111 sentiment lexicon.
    '''
    lex = load_affin()
    f = open(filename, "r")
    line = f.readline()
    while len(line) >0:
        splited = line.split(' ')
        feats1 = {}
        summa = 0
        negga = 0
        pos = 0
        neg = 0
        tot = 0
        
        for word in range(len(splited)-1):
            add = False
            theword = splited[word].split(":")
            lilwords = theword[0].split("_")
            for lw in lilwords:
                if lw in lex:
                    add = True
                    if lex[lw] > 0:
                        summa += lex[lw]*int(theword[1])
                        pos += int(theword[1])
                    else:
                        negga += lex[lw]*int(theword[1])
                        neg += int(theword[1])
            tot += int(theword[1])
        feats1["point_pos"] = summa
        feats1["point_neg"] = -negga
        feats1["count_pos"] = pos
        feats1["count_neg"] = neg
        feats1["size"] = tot
        feats1["porc_pos"] = pos/tot
        feats1["porc_neg"] = neg/tot
        tag = splited[-1].split("#:")[1][0:3]
        thefeatures.append((feats1,tag))
        line = f.readline()
    f.close()

def load_affin():
    '''
    Loads the lexicon of sentiment words included in the AFINN-11 lexicon
    '''
    f = open("AFINN-111.txt", "r")
    line = f.readline()
    affin = {}
    while len(line) >0:
        splited = line.split('\t')
        affin[splited[0]] = int(splited[1])
        line = f.readline()
    return affin

*Additional Function*

Shows the most informative features for the Linear Regression Models.

Taken from the answers to: https://stackoverflow.com/questions/11116697/how-to-get-most-informative-features-for-scikit-learn-classifiers Kudos to him.

In [5]:
def show_most_informative_features(vectorizer, clf, n=20):
    '''
    Gets the most informative features from a Linear Regression Model
    '''
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

**BoW**

Loads the data and creates the feature to train. Loads and uses both review files.

In [6]:
BoWFeatures = []

#Loads the positive reviews
load_bow(BoWFeatures,"reviews/kitchen/positive.review")

#Loads the negative reviews 
load_bow(BoWFeatures,"reviews/kitchen/negative.review")

print("Loaded",len(BoWFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [7]:
BoWTesting =[]

#Loads the testing reviews 
load_bow(BoWTesting,"reviews/kitchen/unlabeled.review")

print("Loaded",len(BoWTesting),"reviews")

Loaded 5945 reviews


**Binary BoW**

Loads the data and creates the feature to train. Loads and uses both review files. Filters them to change it to 1 for each existent element in the Bag of Words.

In [8]:
BinaryBoWFeatures = []

#Loads the positive reviews
load_binary_bow(BinaryBoWFeatures,"reviews/kitchen/positive.review")

#Loads the negative reviews 
load_binary_bow(BinaryBoWFeatures,"reviews/kitchen/negative.review")

print("Loaded",len(BinaryBoWFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [9]:
BinaryBoWTesting =[]

#Loads the testing reviews 
load_binary_bow(BinaryBoWTesting,"reviews/kitchen/unlabeled.review")

print("Loaded",len(BinaryBoWTesting),"reviews")

Loaded 5945 reviews


**Lexicon Features**

Loads the data and creates the feature to train. Loads and uses both review files. Takes into use the following Features:

- Number of positive words
- Number of negative words
- Positive score following the AFINN-111 scores
- Negative score following the AFINN-111 scores
- Size (number of words) of the review
- Percentage of positive words
- Percentage of negative words

In [10]:
LexiconFeatures = []

#Loads the positive reviews
load_lexicon(LexiconFeatures,"reviews/kitchen/positive.review")

#Loads the negative reviews 
load_lexicon(LexiconFeatures,"reviews/kitchen/negative.review")

print("Loaded",len(LexiconFeatures),"reviews")

Loaded 2000 reviews


Loads the data and creates the feature to test the classifier. 

In [11]:
LexiconTesting =[]

#Loads the testing reviews 
load_lexicon(LexiconTesting,"reviews/kitchen/unlabeled.review")

print("Loaded",len(LexiconTesting),"reviews")

Loaded 5945 reviews


**Testing Zone**

Now we are gonna test the six different combinations of Features and classification algorithms

**BoW - Naive Bayes**

Trains the dataset over a Naive Bayes with a Bag of Words Model.

**Training**

In [12]:
shuffle(BoWFeatures)
classifier = nltk.NaiveBayesClassifier.train(BoWFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
              your_money = 1                 neg : pos    =     35.7 : 1.0
              waste_your = 1                 neg : pos    =     30.3 : 1.0
                    easy = 2                 pos : neg    =     26.3 : 1.0
       very_disappointed = 1                 neg : pos    =     22.3 : 1.0
                 awesome = 1                 pos : neg    =     21.0 : 1.0
                   waste = 1                 neg : pos    =     19.3 : 1.0
                 not_buy = 1                 neg : pos    =     19.0 : 1.0
                 easy_to = 2                 pos : neg    =     18.3 : 1.0
               love_this = 1                 pos : neg    =     17.3 : 1.0
                returned = 1                 neg : pos    =     15.6 : 1.0


**Validating**

In [13]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.9985
Precision: 0.998001998001998
Recall: 0.999
F1 Score: 0.9985007496251875


**Testing**

In [14]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8785534062237174
Precision: 0.8932346723044398
Recall: 0.8581584292484766
F1 Score: 0.8753453038674034


**BoW - Linear Regression**

Trains the dataset over a Linear Regression with a Bag of Words Model.

**Training**

In [15]:
shuffle(BoWFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(BoWFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [16]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [17]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8772077375946173
Precision: 0.8779741672331747
Recall: 0.8744075829383886
F1 Score: 0.8761872455902306


In [18]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-1.1427	disappointed   		1.4160	great          
	-1.0509	not            		1.1200	easy           
	-0.8586	poor           		1.0313	best           
	-0.7247	too            		1.0042	love           
	-0.7042	return         		1.0012	excellent      
	-0.5943	broken         		0.9380	perfect        
	-0.5803	broke          		0.8384	works          
	-0.5785	disappointment 		0.7886	easy_to        
	-0.5689	doesn't        		0.6745	well           
	-0.5678	waste          		0.6228	good           


**Binary BoW - Naive Bayes**

Trains the dataset over a Naive Bayes with a Binary Bag of Words Model.

**Training**

In [19]:
shuffle(BinaryBoWFeatures)
classifier = nltk.NaiveBayesClassifier.train(BinaryBoWFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
              your_money = 1                 neg : pos    =     37.0 : 1.0
              waste_your = 1                 neg : pos    =     30.3 : 1.0
       very_disappointed = 1                 neg : pos    =     23.0 : 1.0
                 awesome = 1                 pos : neg    =     21.0 : 1.0
                 not_buy = 1                 neg : pos    =     20.3 : 1.0
               defective = 1                 neg : pos    =     18.3 : 1.0
               love_this = 1                 pos : neg    =     18.1 : 1.0
                   loves = 1                 pos : neg    =     17.7 : 1.0
                returned = 1                 neg : pos    =     17.3 : 1.0
                  refund = 1                 neg : pos    =     17.0 : 1.0


**Validating**

In [20]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.999
Precision: 0.999
Recall: 0.999
F1 Score: 0.9989999999999999


**Testing**

In [21]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.880067283431455
Precision: 0.8932958932958933
Recall: 0.8615436696005416
F1 Score: 0.8771325176632775


**Binary BoW - Linear Regression**

Trains the dataset over a Linear Regression with a Binary Bag of Words Model.

**Training**

In [22]:
shuffle(BinaryBoWFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(BinaryBoWFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [23]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


**Testing**

In [24]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(BinaryBoWTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.8809083263246426
Precision: 0.88406292749658
Recall: 0.8750846310088016
F1 Score: 0.8795508676420551


In [25]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-1.3309	not            		1.5681	great          
	-1.2267	disappointed   		1.1500	easy           
	-0.8215	poor           		1.0546	love           
	-0.7691	too            		1.0493	best           
	-0.6934	broke          		1.0441	excellent      
	-0.6917	after          		0.9789	perfect        
	-0.6860	return         		0.8418	works          
	-0.6291	returned       		0.8200	easy_to        
	-0.6082	waste          		0.7031	good           
	-0.6058	disappointment 		0.6077	well           


**Lexicon Features - Naive Bayes**

Trains the dataset over a Naive Bayes with a Lexicon Features Model.

**Training**

In [26]:
shuffle(LexiconFeatures)
classifier = nltk.NaiveBayesClassifier.train(LexiconFeatures)
classifier.show_most_informative_features(10)

Most Informative Features
               count_pos = 16                pos : neg    =     13.0 : 1.0
               count_neg = 14                neg : pos    =     11.8 : 1.0
               count_pos = 10                pos : neg    =      9.0 : 1.0
               count_neg = 11                neg : pos    =      8.6 : 1.0
               point_neg = 5                 neg : pos    =      8.3 : 1.0
               count_pos = 0                 neg : pos    =      7.9 : 1.0
                porc_pos = 0.0               neg : pos    =      7.9 : 1.0
               point_pos = 0                 neg : pos    =      7.9 : 1.0
               count_neg = 4                 neg : pos    =      7.7 : 1.0
                    size = 33                pos : neg    =      7.0 : 1.0


**Validating**

In [27]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.89
Precision: 0.8793774319066148
Recall: 0.904
F1 Score: 0.8915187376725837


**Testing**

In [28]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.7312026913372582
Precision: 0.720703125
Recall: 0.7494922139471902
F1 Score: 0.7348157982077663


**Lexicon Features - Linear Regression**

Trains the dataset over a Linear Regression with a Lexicon Features Model.

**Training**

In [29]:
shuffle(LexiconFeatures)
classifier = nltk.classify.SklearnClassifier(LogisticRegression(max_iter=1000))
classifier.train(LexiconFeatures)

<SklearnClassifier(LogisticRegression(max_iter=1000))>

**Validating**

In [30]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconFeatures):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For Validating set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For Validating set:
Accuracy: 0.771
Precision: 0.7811203319502075
Recall: 0.753
F1 Score: 0.7668024439918534


**Testing**

In [31]:
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)
real = []
result = []

for i, (feats, label) in enumerate(LexiconTesting):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)
    real.append(label)
    result.append(observed)

print( 'For testing set:')
print( 'Accuracy:', nltk.accuracy(real,result) )
print( 'Precision:', nltk.scores.precision(refsets['pos'], testsets['pos']) )
print( 'Recall:', nltk.recall(refsets['pos'], testsets['pos']) )
print( 'F1 Score:', nltk.f_measure(refsets['pos'], testsets['pos']) )

For testing set:
Accuracy: 0.7613120269133726
Precision: 0.7530497856907352
Recall: 0.7731888964116452
F1 Score: 0.7629864706864874


In [32]:
show_most_informative_features(classifier._vectorizer,classifier._clf,10)  

	-1.4461	porc_neg       		3.1889	porc_pos       
	-0.0875	point_neg      		0.0735	point_pos      
	-0.0168	count_pos      		0.0003	count_neg      
	-0.0019	size           		-0.0019	size           
	0.0003	count_neg      		-0.0168	count_pos      
	0.0735	point_pos      		-0.0875	point_neg      
	3.1889	porc_pos       		-1.4461	porc_neg       


**Table of Results - Kitchen**

| Combination | Accuracy | Precision | Recall | F1 Score|
| --- | ----------- |----------- |----------- |----------- |
| **BOW - NB** | 0.8785534062237174 | 0.8932346723044398 | 0.8581584292484766 | 0.8753453038674034 |
| **BOW - LR** | 0.8772077375946173 | 0.8779741672331747 | 0.8744075829383886 | 0.8761872455902306 |
| **BBOW - NB** | 0.880067283431455 | **0.8932958932958933** | 0.8615436696005416 | 0.8771325176632775 |
| **BBOW - LR** | **0.8809083263246426** | 0.88406292749658 | **0.8750846310088016** | **0.8795508676420551** |
| **Lexicon - NB** | 0.7312026913372582 | 0.720703125 | 0.7494922139471902 | 0.7348157982077663 |
| **Lexicon - LR** | 0.7613120269133726 | 0.7530497856907352 | 0.7731888964116452 | 0.7629864706864874 |
